# Représentations MIDI

Dans la suite de ce notebook, on va essayer d'utiliser miditok pour, d'une part, créer sa propre représentation symbolique des fichiers midis, et d'autre part utiliser les représentations existantes.

Fonctionnement du worklow MidiTok :

- **Preprocessing du midiFile** : le temps est downsampled à la résolution du tokenizer, les tracks sont fusionnés, les pitches en dehors du tokenizer sont enlevés, les notes et les tempos sont downsampled et les notes, le tempo et la time signature sont dédupliqués.

- **Parsing des évenements globaux** : Token de tempos et de signatures temporels sont créés.

- **Parsing des évenements des pistes** : Les notes, les accords, les contrôles (pédales...) et les tokens spécifiques à chaque piste sont analysés pour créer les tokens qui leur sont associés.

- **Creating time tokens** : Les tokens feprésentant le temps sont créés afin de lier les jetons globaux et de suivi précédemment créés.


### Représentation issue de l'article :"Using Machine-Learning Methods for Musical Style Modeling"

Toolkit de 5 filtre simplificateurs :
- *Filtre d'arpèges* : Aligne verticalement les notes qui ont été attaqués quasi en même temps.

- *Filtre legato* : Supprime le chevauchement entre deux notes successives.

- *Filtre stacato* : Ignore les silences entre les notes successives.

- *Filtre de diffusion* : Aligne verticalement les diffusions de notes 

- *Filtre de durée* : Quantifie statistiquement les durées pour réduire l'alphabet de durée


```lien vers la documentation```  : https://miditok.readthedocs.io/en/latest/tokenizations.html 

In [1]:
# Imports
from mido import MidiFile, MidiTrack, Message
import mido
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
def extract_features(midi_file, output='nested_list'):
    """
    Extrait les paramètres de note d'un fichier MIDI et renvoie une matrice.
    
    Args:
        midi_file: Chemin vers le fichier MIDI.
        output: Format de sortie, 'nested_list' ou 'polars'.
        
    Returns:
        Une matrice contenant les notes avec colonnes [pitch, onset, duration].
    """
    try:
        # Charger le fichier MIDI
        mid = mido.MidiFile(midi_file)
        notes = []
        active_notes = {}  # clé: (track, channel, note) -> valeur: {'start_time': ..., 'velocity': ...}

        # Parcourir toutes les pistes du fichier MIDI
        for track_index, track in enumerate(mid.tracks):
            current_time = 0
            for msg in track:
                current_time += msg.time

                # Ignorer les messages liés au tempo ou à la signature temporelle
                if msg.type in ['set_tempo', 'time_signature']:
                    continue

                # Note-on (activation)
                if msg.type == 'note_on' and msg.velocity > 0:
                    key = (track_index, msg.channel, msg.note)
                    active_notes[key] = {
                        'start_time': current_time,
                        'velocity': msg.velocity
                    }
                # Note-off (ou note_on avec vélocité nulle)
                elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                    key = (track_index, msg.channel, msg.note)
                    if key in active_notes:
                        start_info = active_notes.pop(key)
                        start_time = start_info['start_time']
                        duration_ticks = current_time - start_time
                        # On ne conserve que pitch, onset et duration
                        notes.append({
                            'pitch': msg.note,
                            'onset': start_time,
                            'duration': duration_ticks,
                            'velocity': start_info['velocity']
                        })

        # Trier les notes par onset (temps de début)
        notes = sorted(notes, key=lambda x: x['onset'])
        # Construire la matrice sous forme de liste de listes
        mnotes = [[n['pitch'], n['onset'], n['duration'], n['velocity']] for n in notes]

        if output == 'nested_list':
            return mnotes
        elif output == 'polars':
            return pl.DataFrame(mnotes, schema=['pitch', 'onset', 'duration', 'velocity'])
        else:
            raise ValueError("Le paramètre output doit être 'nested_list' ou 'polars'")
    except Exception as e:
        print("Erreur lors de l'extraction:", e)
        return None
    



In [3]:
"""
Petit code snipet sur la quantization
"""

def quantize_value(value, stepSize):
    
    # Retourne une valeur arroondie au multiple le plus proche de stepSize
    return np.round(value/stepSize) * stepSize

def quantize_matrix(matrix, stepSize=10, quantizeOnsets=True, quantizeDurations=True):
    """
    Applique plusieurs filtres sur une matrice d'événements MIDI.
    
    Paramètres :
      matrix : np.ndarray de forme (N, >=2) où
               - colonne 0 = onset
               - colonne 1 = offset (release)
      stepSize : unité de quantification (par défaut 10)
      quantizeOnsets : si True, quantifie également les offsets
      quantizeDurations : si True, quantifie la durée des notes
      
    Retourne :
      La matrice quantifiée et filtrée.
    """
    if isinstance(matrix,pl.DataFrame):
      mat = matrix.clone().to_numpy()

    else:
       mat = np.array(matrix.copy())

    

    # On aligne le début de chaque note sur une grille de steSize = stepSize (par défaut 10ms)
    if quantizeOnsets:
      for i in range(len(mat)):
        mat[i, 1] = quantize_value(mat[i, 1], stepSize=stepSize)

    # On aligne les durations sur la grille
    if quantizeDurations:
      for i in range(len(mat)):
        mat[i, 2] = quantize_value(mat[i, 2], stepSize=stepSize)

    # On calcule l'offset après l'onset et la duration quantifié
    offset = mat[:,1] + mat[:,2]

    mat = pl.DataFrame(mat)
    return mat

In [ ]:
midFile = '/home/sylogue/Documents/MuseScore4/Scores/o clair de la lune.mid'
mid1 = extract_features(midFile, "polars")

with open("midiJson.json", "w") as midiJson: 
    a =mid1.write_ndjson()
    json.dump(a, midiJson)


/tmp/ipykernel_55869/530720673.py:58: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return pl.DataFrame(mnotes, schema=['pitch', 'onset', 'duration', 'velocity'])


In [5]:
# Improvisation


def NaiveImpro(symbols)

SyntaxError: expected ':' (2017585293.py, line 4)